# App showing the results from our fine tuned GPT-2 model

To run this notebook (app) you need to add the image files (you can add your own images, they are used to create the icon for the tweet images), and the arial.ttf file since it is used for defining the font of which the tweets will be written in. 


In [1]:
!pip install gradio
!pip install hopsworks

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import gradio as gr
import hopsworks
import pandas as pd
import numpy as np
from PIL import Image, ImageDraw, ImageFont
from textwrap import wrap



def create_tweet_img(text, img_file, user):
  # Constants
  # -----------------------------------------------------------------------------
  # Set the font to be used 
  FONT_USER_INFO = ImageFont.truetype("arial.ttf", 90, encoding="utf-8")
  FONT_TEXT = ImageFont.truetype("arial.ttf", 110, encoding="utf-8")
  # Image dimensions (pixels)
  WIDTH = 2376
  HEIGHT = 2024
  # Color scheme
  COLOR_BG = 'white'
  COLOR_NAME = 'black'
  COLOR_TAG = (64, 64, 64)
  COLOR_TEXT = 'black'
  # Write coordinates
  COORD_PHOTO = (250, 170)
  COORD_NAME = (600, 185)
  COORD_TAG = (600, 305)
  COORD_TEXT = (250, 510)
  # Extra space to add in between lines of text
  LINE_MARGIN = 15
  # -----------------------------------------------------------------------------


  # Information for the image
  # -----------------------------------------------------------------------------
  if user == 0:
    user_name = "Elon Musk GPT2"
    user_tag = "@elonmusk-gpt2"
    text = text
    img_name = "test"
  if user == 1:
    user_name = "Elon Musk"
    user_tag = "@elonmusk"
    text = text
    img_name = "test"
  # -----------------------------------------------------------------------------

  # Setup of variables and calculations
  # -----------------------------------------------------------------------------
  # Break the text string into smaller strings, each having a maximum of 37\
  # characters (a.k.a. create the lines of text for the image)
  text_string_lines = wrap(text, 37)

  # Horizontal position at which to start drawing each line of the tweet body
  x = COORD_TEXT[0]

  # Current vertical position of drawing (starts as the first vertical drawing\
  # position of the tweet body)
  y = COORD_TEXT[1]

  # Create an Image object to be used as a means of extracting the height needed\
  # to draw each line of text
  temp_img = Image.new('RGB', (0, 0))
  temp_img_draw_interf = ImageDraw.Draw(temp_img)

  # List with the height (pixels) needed to draw each line of the tweet body
  # Loop through each line of text, and extract the height needed to draw it,\
  # using our font settings
  line_height = [
      temp_img_draw_interf.textsize(text_string_lines[i], font=FONT_TEXT)[1]
      for i in range(len(text_string_lines))
  ]
  # -----------------------------------------------------------------------------

  # Image creation
  # -----------------------------------------------------------------------------
  # Create what will be the final image
  img = Image.new('RGB', (WIDTH, HEIGHT), color='white')
  # Create the drawing interface
  draw_interf = ImageDraw.Draw(img)

  # Draw the user name
  draw_interf.text(COORD_NAME, user_name, font=FONT_USER_INFO, fill=COLOR_NAME)
  # Draw the user handle
  draw_interf.text(COORD_TAG, user_tag, font=FONT_USER_INFO, fill=COLOR_TAG)

  # Draw each line of the tweet body. To find the height at which the next\
  # line will be drawn, add the line height of the next line to the current\
  # y position, along with a small margin
  for index, line in enumerate(text_string_lines):
      # Draw a line of text
      draw_interf.text((x, y), line, font=FONT_TEXT, fill=COLOR_TEXT)
      # Increment y to draw the next line at the adequate height
      y += line_height[index] + LINE_MARGIN

  # Load the user photo (read-mode). It should be a 250x250 circle 

  # load image
  im = Image.open(img_file)
  im = im.resize((250, 250))

  # Resize the image
  im = im.resize((250, 250))

  # Create a circular mask
  mask = Image.new('L', (250, 250), 0)
  draw = ImageDraw.Draw(mask) 
  draw.ellipse((0, 0) + (250, 250), fill=255)

  # Apply the mask to the image
  im.putalpha(mask)


  # Paste the user photo into the working image. We also use the photo for\
  # its own mask to keep the photo's transparencies
  img.paste(im, COORD_PHOTO, mask=im)

  # Finally, save the created image
  return img
  # ----------------------------

project = hopsworks.login()
fs = project.get_feature_store()

monitor_fg = fs.get_feature_group(name="em_gpt_monitor", version=1)
monitor_df = monitor_fg.read()

original_tweet = monitor_df["original_tweet"].iloc[-1]
predicted_tweet = monitor_df["predicted_tweet"].iloc[-1]
random_tweet = monitor_df["random_tweet"].iloc[-1]


original_tweet_image = create_tweet_img(original_tweet, "emprofilepic.jpg", 1)
predicted_tweet_image = create_tweet_img(predicted_tweet, "twitterprofilepicgpt2.png", 0)
random_tweet_image = create_tweet_img(random_tweet, "twitterprofilepicgpt2.png", 0)

original_im = original_tweet_image.convert("RGB")
predicted_im = predicted_tweet_image.convert("RGB")
random_im = random_tweet_image.convert("RGB")

with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
          gr.Label("Today's Tweet by Elon Musk")
          input_img = gr.Image(original_im, elem_id="original-img")
        with gr.Column():
          gr.Label("Today's Predicted Tweet Using The Original Tweet by Elon Musk")
          input_img = gr.Image(predicted_im, elem_id="Predicted_img")
    with gr.Row():
        with gr.Column():
          gr.Label("Today's Random Tweet Generated by GPT-2")
          input_img = gr.Image(random_im, elem_id="Random_img")
demo.launch()


Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/4251
Connected. Call `.close()` to terminate connection gracefully.
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>